In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install bitsandbytes

In [ ]:
!pip install -U bitsandbytes transformers

In [ ]:
!pip install faiss-gpu-cu12

In [ ]:
!pip install -U langchain-community


In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import numpy as np
import torch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
data_path = "/content/drive/MyDrive/"

In [ ]:
train = pd.read_csv(data_path+"train.csv" )
test = pd.read_csv(data_path+"test.csv" )
sample = pd.read_csv(data_path+"sample_submission.csv")

In [ ]:
def preprocess(df):
    df.replace('-', np.nan, inplace=True)
    df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
    df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
    df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
    df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
    df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
    df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]
    df['사고인지 시간'] = df['사고인지 시간'].str.split('-').str[0].str.strip()

    return df


train = preprocess(train)
test = preprocess(test)


In [ ]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"작업 프로세스는 '{row['작업프로세스']}'이며 {row['인적사고']}발생, 사고 원인은 '{row['사고원인']}'입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [ ]:
# 훈련 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"작업 프로세스는 '{row['작업프로세스']}'이며 {row['인적사고']}발생, 사고 원인은 '{row['사고원인']}'입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        )
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto"
)

In [ ]:
import torch
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
from tqdm import tqdm

# Train 데이터 준비
train_questions_prevention = combined_training_data['question'].tolist()
train_answers_prevention = combined_training_data['answer'].tolist()

train_documents = [
    f"Q: {q1}\nA: {a1}"
    for q1, a1 in zip(train_questions_prevention, train_answers_prevention)
]

# 임베딩 생성
embedding_model_name = "jhgan/ko-sbert-sts"
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 벡터 스토어에 문서 추가
vector_store = FAISS.from_texts(train_documents, embedding)

# Retriever 정의
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})






In [ ]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.1, # 0~1 까지 0에 가까울 수록 보수적인 답변 즉 창의성이 떨어짐
    return_full_text=False,
    max_new_tokens=128, # 문장 최대 길이 조정
    batch_size=16  # 배치 크기 지정
)

In [ ]:
prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 **재발 방지 대책 및 향후 조치 계획**만 간결하게 답변하세요.
- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.
- 불필요한 형식(목차, 강조 표시, 리스트 등)을 사용하지 마세요.
- 한 문장 또는 간결한 문단으로 자연스럽게 작성하세요.
- 특수문자를 포함하지 마세요.

{context}

### 질문:
{question}

### 답변:
"""

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 커스텀 프롬프트 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
tokenizer.padding_side = "left"

In [ ]:
from torch.utils.data import DataLoader
from datasets import Dataset
import torch
from tqdm import tqdm
import psutil

# Matmul 성능 최적화
torch.set_float32_matmul_precision('high')

# 테스트 데이터 준비
test_questions = combined_test_data['question'].tolist()
test_dataset = Dataset.from_dict({"question": test_questions})

# DataLoader 설정
batch_size = 16
num_workers = 0  # GPU 환경에서는 num_workers=0이 적절함
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

test_results = []
print("테스트 실행 시작... 총 테스트 샘플 수:", len(test_questions))

# ✅ apply()를 활용한 배치 처리 함수
def process_batch(batch):
    batch_inputs = [{"question": q} for q in batch["question"]]  # apply()를 위한 입력 형태 변환
    batch_outputs = qa_chain.apply(batch_inputs)  # LangChain을 통한 배치 처리
    return batch_outputs  # 출력값 리스트 반환

# DataLoader를 이용한 배치 처리
for batch in tqdm(test_dataloader, desc="처리 진행률"):
    batch_results = process_batch(batch)
    test_results.extend(batch_results)  # 결과 누적

print("\n테스트 실행 완료! 총 결과 수:", len(test_results))


In [ ]:
test_results

In [ ]:
test_results[0]

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

In [ ]:
submission = pd.read_csv(data_path+"sample_submission.csv", encoding = 'utf-8-sig')
# 최종 결과 저장
submission.iloc[:,1] = test_results
submission.iloc[:,2:] = pred_embeddings
submission.head()
# 최종 결과를 CSV로 저장
submission.to_csv('submission-EXAONE-3.5-32B-Instruct.csv', index=False, encoding='utf-8-sig')